## Imports

In [1]:
import itertools

import networkx as nx
import numpy as np

from aoc_utilities import puzzle_input

<h2>--- Day 15: Chiton ---</h2><p>You've almost reached the exit of the cave, but the walls are getting closer together. Your submarine can barely still fit, though; the main problem is that the walls of the cave are covered in <a href="https://en.wikipedia.org/wiki/Chiton" target="_blank">chitons</a>, and it would be best not to bump any of them.</p>
<p>The cavern is large, but has a very low ceiling, restricting your motion to two dimensions. The shape of the cavern resembles a square; a quick scan of chiton density produces a map of <em>risk level</em> throughout the cave (your puzzle input). For example:</p>
<pre><code>1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
</code></pre>
<p>You start in the top left position, your destination is the bottom right position, and you <span title="Can't go diagonal until we can repair the caterpillar unit. Could be the liquid helium or the superconductors.">cannot move diagonally</span>. The number at each position is its <em>risk level</em>; to determine the total risk of an entire path, add up the risk levels of each position you <em>enter</em> (that is, don't count the risk level of your starting position unless you enter it; leaving it adds no risk to your total).</p>
<p>Your goal is to find a path with the <em>lowest total risk</em>. In this example, a path with the lowest total risk is highlighted here:</p>
<pre><code><em>1</em>163751742
<em>1</em>381373672
<em>2136511</em>328
369493<em>15</em>69
7463417<em>1</em>11
1319128<em>13</em>7
13599124<em>2</em>1
31254216<em>3</em>9
12931385<em>21</em>
231194458<em>1</em>
</code></pre>
<p>The total risk of this path is <code><em>40</em></code> (the starting position is never entered, so its risk is not counted).</p>
<p><em>What is the lowest total risk of any path from the top left to the bottom right?</em></p>


## Solution
This amounts to computing a shortest path algorithm on the directed graph $R$ with
1. a node for each "pixel" in the risk map of the cave,
2. directed edges $p_1 \to p_2$ between each pair $p_1$ and $p_2$ of adjacent pixels, and
3. the weight on each edge with target pixel $p$ the risk of entering pixel $p$.

NetworkX again makes this a few lines of code.
To make things more interesting, I've written a solution that will work for any box-shaped cave (unequal side lengths permitted) in any number of dimensions.
You know … just in case you're planning to do any seven-dimensional submarine spelunking in the future.

In [2]:
def lowest_risk_path_length(risks: np.ndarray):
    R = nx.DiGraph()
    cavern_dimensions = np.array(risks.shape)

    for hypervoxel in itertools.product(*map(range, cavern_dimensions)):
        for neighbor in get_neighbors(hypervoxel, risks.shape):
            R.add_edge(neighbor, hypervoxel, risk=risks[hypervoxel])
            R.add_edge(hypervoxel, neighbor, risk=risks[neighbor])

    return nx.shortest_path_length(
        R, (0, 0), tuple(cavern_dimensions - 1), weight='risk')


def get_neighbors(index, shape):
    dimension_count = len(shape)
    for dimension in range(dimension_count):
        index_entry = index[dimension]
        if index_entry > 0:
            yield (*index[:dimension], index_entry - 1, *index[dimension + 1:])
        if index_entry < shape[dimension] - 1:
            yield (*index[:dimension], index_entry + 1, *index[dimension + 1:])

### Testing

In [3]:
example_risks = np.array([
    [int(risk) for risk in line]
    for line in '''
        1163751742
        1381373672
        2136511328
        3694931569
        7463417111
        1319128137
        1359912421
        3125421639
        1293138521
        2311944581
    '''.split()
])

assert lowest_risk_path_length(example_risks) == 40

### Answer

In [4]:
risks = np.array([
    [int(risk) for risk in line]
    for line in puzzle_input.as_text(day=15).split()
])

lowest_risk_path_length(risks)

403